In [3]:
import pandas as pd
import numpy as np
import datetime
import random

# ------------------ MASTER INTENSITY PROFILE (Simplified) ------------------
def generate_master_intensity_profile():
    """Returns an empty list as the master profile is no longer used."""
    return []

# ------------------ STORM CELL GENERATION (Modified) ------------------
def generate_storm_cell_lifecycle(
    cell_id,
    start_date,
    end_date,
    target_peak_intensity_dbz,  # Now a variable passed to the function
    fixed_lifetime_hours,       # New parameter for fixed lifetime
    time_step_minutes=5,
    base_size_pixels=120,
    base_vx=0.0,
    base_vy=2.0,
    movement_randomness_scale=0.2
):
    """
    Generates a single storm cell's lifecycle with a fixed lifetime and
    a unique peak intensity for each storm.
    """
    records = []

    # Random formation time
    time_range_seconds = (end_date - start_date).total_seconds()
    formation_time = start_date + datetime.timedelta(seconds=random.uniform(0, time_range_seconds))

    # --- NEW: Fixed Lifetime ---
    lifetime_hours = fixed_lifetime_hours
    lifetime_delta = datetime.timedelta(hours=lifetime_hours)
    dissipation_time = formation_time + lifetime_delta

    # Starting position
    x_position, y_position = 0, 0

    previous_intensity = 0
    peak_relative_time = 0.5 

    current_time = formation_time
    while current_time <= dissipation_time:
        time_elapsed_seconds = (current_time - formation_time).total_seconds()
        
        # Normalize time elapsed to a 0-1 range
        progress = time_elapsed_seconds / lifetime_delta.total_seconds()
        
        # --- Fixed peak intensity logic is now per-storm ---
        if abs(progress - peak_relative_time) < (time_step_minutes / 60) / (lifetime_hours * 2):
            intensity_dbz = target_peak_intensity_dbz
        elif progress < peak_relative_time:
            intensity_dbz = target_peak_intensity_dbz / (1 + np.exp(-10 * (progress - peak_relative_time)))
        else:
            decay_progress = (progress - peak_relative_time) / (1 - peak_relative_time)
            intensity_dbz = target_peak_intensity_dbz * np.exp(-10 * decay_progress)

        intensity_dbz = max(0.0, intensity_dbz)
        
        intensity_change_rate = (intensity_dbz - previous_intensity) / (time_step_minutes / 60)

        size_pixels = 20 + (intensity_dbz / target_peak_intensity_dbz) * base_size_pixels if target_peak_intensity_dbz > 0 else 20
        size_pixels = size_pixels * random.uniform(0.9, 1.1)
        size_pixels = max(0, int(size_pixels))

        # --- FIXED: Rainfall calculation with multiplicative noise (±10%) ---
        base_rainfall = 0.08 * (intensity_dbz ** 1.5)
        rainfall_factor = 1.0 + random.uniform(-0.1, 0.1)  # ±10%
        rainfall_mmhr = base_rainfall * rainfall_factor
        rainfall_mmhr = max(0.0, rainfall_mmhr)

        if intensity_dbz == 0.0:
            rainfall_mmhr = 0.0

        # Movement (unchanged)
        vx = base_vx + np.random.normal(0, movement_randomness_scale)
        vy = base_vy + np.random.normal(0, movement_randomness_scale)
        x_position += vx * (time_step_minutes / 60)
        y_position += vy * (time_step_minutes / 60)

        records.append({
            'cell_id': cell_id,
            'timestamp_utc': current_time,
            'formation_time_utc': formation_time,
            'dissipation_time_utc': dissipation_time,
            'lifetime_hours': lifetime_hours,
            'time_since_formation_hours': time_elapsed_seconds / 3600,
            'x_position': x_position,
            'y_position': y_position,
            'size_pixels': size_pixels,
            'intensity_dbz': intensity_dbz,
            'rainfall_mm_per_hr': rainfall_mmhr,
            'intensity_change_rate': intensity_change_rate
        })

        previous_intensity = intensity_dbz
        current_time += datetime.timedelta(minutes=time_step_minutes)

    return records


# ------------------ RUN SIMULATION ------------------
print("--- Generating storms with a fixed lifetime of 1.5 hours and unique peak intensity per storm ---")

all_storm_data = []
num_simulated_cells = 50000
overall_start_date = datetime.datetime(2024, 8, 1, 0, 0, 0)
overall_end_date = datetime.datetime(2024, 8, 5, 23, 59, 59)

fixed_storm_lifetime = 1.5 # Fixed lifetime in hours

# Define a distribution for the unique peak intensities
# Mean of 40 dBZ, standard deviation of 10, clamped between 10 and 70 dBZ
mean_peak_intensity = 40
std_dev_peak_intensity = 10
min_peak_intensity = 10
max_peak_intensity = 70

for i in range(num_simulated_cells):
    # --- NEW: Generate a unique peak intensity for each storm ---
    unique_peak_intensity = np.random.normal(mean_peak_intensity, std_dev_peak_intensity)
    unique_peak_intensity = max(min_peak_intensity, min(max_peak_intensity, unique_peak_intensity))
    
    cell_records = generate_storm_cell_lifecycle(
        cell_id=f'StormCell_{i+1:05d}',
        start_date=overall_start_date,
        end_date=overall_end_date,
        target_peak_intensity_dbz=unique_peak_intensity,
        fixed_lifetime_hours=fixed_storm_lifetime,
        base_vx=0.0,
        base_vy=2.0,
        movement_randomness_scale=0.2
    )
    all_storm_data.extend(cell_records)

# Create DataFrame
storm_df = pd.DataFrame(all_storm_data)
storm_df = storm_df.sort_values(by=['cell_id', 'timestamp_utc']).reset_index(drop=True)

# Save CSV
output_file = "scenario7_fixed_lifetime_unique_intensity_50000.csv"
storm_df.to_csv(output_file, index=False, date_format='%Y-%m-%d %H:%M:%S.%f')

print(f"Generated {len(storm_df)} rows for {num_simulated_cells} storms → saved to {output_file}.")


--- Generating storms with a fixed lifetime of 1.5 hours and unique peak intensity per storm ---
Generated 950000 rows for 50000 storms → saved to scenario7_fixed_lifetime_unique_intensity_50000.csv.


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# -------------------- Load and Inspect Data --------------------
# This script assumes the CSV file is in the same directory.
# If you get a FileNotFoundError, please ensure the file is in the same directory as this script.
file_path = "scenario7_fixed_lifetime_unique_intensity_50000.csv"
try:
    storm_df = pd.read_csv(file_path)
    print("Data loaded successfully.")
except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found. Please ensure the file is in the correct directory.")
    # Exit the script gracefully if the file is not found
    exit()

# Convert relevant columns to appropriate data types
storm_df['timestamp_utc'] = pd.to_datetime(storm_df['timestamp_utc'])
storm_df['formation_time_utc'] = pd.to_datetime(storm_df['formation_time_utc'])
storm_df['dissipation_time_utc'] = pd.to_datetime(storm_df['dissipation_time_utc'])

# -------------------- 1. Verify Fixed Lifetime --------------------
print("\n--- Verifying Fixed Lifetime ---")
# Get a single lifetime value for each storm cell
lifetimes = storm_df.groupby('cell_id')['lifetime_hours'].first()

# Check if the minimum and maximum lifetimes are the same
min_lifetime = lifetimes.min()
max_lifetime = lifetimes.max()
num_unique_lifetimes = lifetimes.nunique()

print(f"Number of unique lifetimes found: {num_unique_lifetimes}")
print(f"Minimum lifetime: {min_lifetime:.2f} hours")
print(f"Maximum lifetime: {max_lifetime:.2f} hours")

if num_unique_lifetimes == 1 and np.isclose(min_lifetime, 1.5):
    print("Verification successful: All storms have a fixed lifetime of 1.5 hours.")
else:
    print("Verification failed: Lifetimes are not fixed as expected.")

# -------------------- 2. Verify Unique Peak Intensity --------------------
print("\n--- Verifying Unique Peak Intensity ---")
# Group by storm cell and find the maximum intensity for each
peak_intensities = storm_df.groupby('cell_id')['intensity_dbz'].max()
num_unique_peak_intensities = peak_intensities.nunique()

print(f"Number of storms analyzed: {peak_intensities.shape[0]}")
print(f"Number of unique peak intensities found: {num_unique_peak_intensities}")
print(f"Percentage of unique peak intensities: {num_unique_peak_intensities / peak_intensities.shape[0] * 100:.2f}%")

# Print descriptive statistics to show the variability
print("\nDescriptive Statistics for Peak Intensities:")
print(peak_intensities.describe())

# -------------------- 3. Visualize Two Storm Lifecycles --------------------
print("\n--- Visualizing Two Unique Storms with Different Peak Intensities ---")

# Get the storm with the highest and lowest peak intensity to highlight the difference
min_peak_storm_id = peak_intensities.idxmin()
max_peak_storm_id = peak_intensities.idxmax()

# Filter the data for these two storms
low_intensity_storm_df = storm_df[storm_df['cell_id'] == min_peak_storm_id]
high_intensity_storm_df = storm_df[storm_df['cell_id'] == max_peak_storm_id]

# Create a figure with a 2x1 grid of subplots
fig, axes = plt.subplots(2, 1, figsize=(12, 10), sharex=True)
fig.suptitle('Comparison of Storms with Different Peak Intensities (Fixed Lifetime)', fontsize=16)

# Plot Intensity vs. Lifetime
axes[0].plot(low_intensity_storm_df['time_since_formation_hours'], low_intensity_storm_df['intensity_dbz'], label=f'Low Intensity Storm (Peak: {low_intensity_storm_df["intensity_dbz"].max():.2f} dBZ)')
axes[0].plot(high_intensity_storm_df['time_since_formation_hours'], high_intensity_storm_df['intensity_dbz'], label=f'High Intensity Storm (Peak: {high_intensity_storm_df["intensity_dbz"].max():.2f} dBZ)')
axes[0].set_title('Intensity vs. Time Since Formation')
axes[0].set_xlabel('Time Since Formation (hours)')
axes[0].set_ylabel('Intensity (dBZ)')
axes[0].legend()
axes[0].grid(True)
axes[0].set_xlim(0, 1.5) # Set x-axis limit to show the fixed lifetime

# Plot Rainfall vs. Lifetime
axes[1].plot(low_intensity_storm_df['time_since_formation_hours'], low_intensity_storm_df['rainfall_mm_per_hr'], label=f'Low Intensity Storm')
axes[1].plot(high_intensity_storm_df['time_since_formation_hours'], high_intensity_storm_df['rainfall_mm_per_hr'], label=f'High Intensity Storm')
axes[1].set_title('Rainfall vs. Time Since Formation')
axes[1].set_xlabel('Time Since Formation (hours)')
axes[1].set_ylabel('Rainfall (mm/hr)')
axes[1].legend()
axes[1].grid(True)
axes[1].set_xlim(0, 1.5)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig('storm_lifecycle_comparison.png')
plt.close()

print("\nVisualization saved to 'storm_lifecycle_comparison.png'.")

Data loaded successfully.

--- Verifying Fixed Lifetime ---
Number of unique lifetimes found: 1
Minimum lifetime: 1.50 hours
Maximum lifetime: 1.50 hours
Verification successful: All storms have a fixed lifetime of 1.5 hours.

--- Verifying Unique Peak Intensity ---
Number of storms analyzed: 50000
Number of unique peak intensities found: 49867
Percentage of unique peak intensities: 99.73%

Descriptive Statistics for Peak Intensities:
count    50000.000000
mean        39.980905
std          9.926840
min         10.000000
25%         33.315113
50%         39.949684
75%         46.692241
max         70.000000
Name: intensity_dbz, dtype: float64

--- Visualizing Two Unique Storms with Different Peak Intensities ---

Visualization saved to 'storm_lifecycle_comparison.png'.
